### Method:
from 0.0.0, trying method xyz, using data abc

### Result:
We noted that method xyz took 5 minutes to run and the result is 5% different compared to 0.0.0

In [ ]:
# general imports

import os, sys, re
import numpy as np

def root_dir(project_name):
    end_ind = re.search(project_name, os.getcwd()).end() # finds the last index of the first occurance
    return os.getcwd()[:end_ind]
ROOT_DIR = root_dir('GEM') # MANUAL

SRC_DIR = f'{ROOT_DIR}/src'
sys.path.append(SRC_DIR)
import traceback
import logging

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# %matplotlib notebook
%matplotlib inline 
plt.style.use('ggplot')

import pickle
import csv
from IPython.display import display
from tqdm import tqdm
from pdb import set_trace as bp;

from sdutils.plot import plot
from sdutils.files import ensure_dir
from sdutils.describe import describe

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 0)

%load_ext memory_profiler
%load_ext autoreload
%autoreload

# some weird error while import keras 
# # http://zachmoshe.com/2017/04/03/pickling-keras-models.html
# import types
# import tempfile
# import keras.models

# def make_keras_picklable():
#     def __getstate__(self):
#         model_str = ""
#         with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
#             keras.models.save_model(self, fd.name, overwrite=True)
#             model_str = fd.read()
#         d = { 'model_str': model_str }
#         return d

#     def __setstate__(self, state):
#         with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
#             fd.write(state['model_str'])
#             fd.flush()
#             model = keras.models.load_model(fd.name)
#         self.__dict__ = model.__dict__


#     cls = keras.models.Model
#     cls.__getstate__ = __getstate__
#     cls.__setstate__ = __setstate__

# make_keras_picklable()

import random
import time 
seed = int(time.time())
# print('Warning: MANUAL seed')
# seed = 1519972733 # MANUAL
random.seed(seed)
np.random.seed(seed) # !!!
print("Seed was:", seed)

In [ ]:
%%javascript
// https://stackoverflow.com/questions/12544056/how-do-i-get-the-current-ipython-notebook-name
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
# project imports

from sklearn.model_selection import GridSearchCV, ParameterGrid, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, RandomForestClassifier
import scikitplot as skplt
from sklearn.metrics import classification_report 
from sklearn.dummy import DummyClassifier

from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTETomek

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

import multiprocessing

# sanity check for NUM_TAG and comfirming OUTPUT_DIR
try:
    match = re.search(r'^\d+\.\d+\.\d+', theNotebook)
    NUM_TAG = match.group() # will error if not found
except: 
    print('WARNING: Please check NUM_TAG and set it MANUALLY.')
    NUM_TAG = '1.0.10'
ASSET_DIR = f'{ROOT_DIR}/assets'
OUTPUT_DIR = ensure_dir(f'{ASSET_DIR}/{NUM_TAG}')
print(OUTPUT_DIR)

In [ ]:
# # useful commands

# df_train = pd.read_csv('../data/xxx/file_train.csv')
# df_test = pd.read_csv('../data/xxx/file_test.csv')

# !cat "{OUTPUT_DIR}/0_0_1_experiments_roc_summary.txt"
# !rm "{OUTPUT_DIR}/0_0_1_experiments_roc_summary.txt"

# with open(f"{OUTPUT_DIR}/0_0_1_experiments_roc_summary.txt", "a") as fp:
#     print('hep', file= fp )